In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from pathlib import Path

### rename files

In [3]:
root = "/home/user/Documents/data/sudoku-assistant/raw_data"
files = Path(root).glob('*.jpg')

sorted_files = sorted(list(files), key=lambda x: x.name)

for i, file in enumerate(sorted_files):
    file.rename(file.with_stem(f"sudoku_raw_photo_{i}"))

### visualize

In [4]:


def viz(img, title=''):
    plt.figure()

    if len(img.shape) == 3:
        plt.imshow(img[:, :, ::-1])
    else:
        plt.imshow(img, cmap='gray')
    
    plt.title(title)
    plt.show()


## Preprocessing

In [ ]:
def select_best_countour(cntrs: np.ndarray):
    
    FIELD_ARC_EPSILON = 0.02
    
    sorted_cntrs = sorted(cntrs, key=cv.contourArea, reverse=True)
    assert sorted_cntrs, "No contours found..."

    for cntr in sorted_cntrs:
        epsilon = FIELD_ARC_EPSILON * cv.arcLength(cntr, closed=True)
        corner_points = cv.approxPolyDP(cntr, epsilon, closed=True)
        
        if len(corner_points) == 4:
            return corner_points

def reorder_cntr_corners(points: np.ndarray):

    assert points.shape == (4, 2)

    x_c = points[:, 0].mean()
    y_c = points[:, 1].mean()

    tl = tr = br = bl = 0

    for p in points:
        x, y = p
        if x < x_c and y < y_c:
            tl = p
        elif x > x_c and y < y_c:
            tr = p
        elif x > x_c and y > y_c:
            br = p
        elif x < x_c and y > y_c:
            bl = p

    return (tl, tr, br, bl)

def get_sudoku_field(img_: np.ndarray):

    debug = False

    img = img_.copy()

    # convert to grayscale
    img = cv.cvtColor(img, code=cv.COLOR_BGR2GRAY)
    # slight blur to remove noise
    img = cv.GaussianBlur(img, (5, 5), sigmaX=1)
    # increase contrast

    brightness = 0
    contrast = 1
    brightness += int(round(255*(1-contrast)/2))
    img = cv.addWeighted(img, contrast, img, 0, brightness)
    #img = np.clip(1.25 * img.astype(np.uint16), 0, 255).astype(np.uint8)

    # apply adaptive threshold to binarize image
    img = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 25, 10)
    # find the largest contour
    cntrs, _ = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    best_cntr_corners = select_best_countour(cntrs)

    if debug:
        img_with_main_cntr = img.copy()
        cv.drawContours(img_with_main_cntr, [best_cntr_corners], 0, color=255, thickness=20)


    tl, tr, br, bl = reorder_cntr_corners(best_cntr_corners.squeeze())

    width = max(np.sqrt(np.sum(tl - tr)**2), np.sqrt(np.sum(bl - br)**2))
    height = max(np.sqrt(np.sum(tl - bl)**2), np.sqrt(np.sum(tr-br)**2))


    corners_old = np.array([tl, tr, br, bl], dtype=np.float32)
    corners_new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

    matrix = cv.getPerspectiveTransform(corners_old, corners_new)

    img = cv.warpPerspective(img, matrix, (int(width), int(height)))

    cv.rectangle(img, pt1=(0, 0), pt2=(int(width), int(height)), color=255, thickness=5)

    img = cv.morphologyEx(img, cv.MORPH_CLOSE, np.ones((3, 3), np.uint8))

    return img


def get_lines_mask(img):

    lines = cv.HoughLinesP(img, 1, np.pi / 180, 200, minLineLength=int(img.shape[0] / 9 * 0.80), maxLineGap=5)

    # Create a mask for the lines
    mask = np.zeros_like(img)

    cell_width = img.shape[1] / 9
    cell_height = img.shape[0] / 9


    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]  # Extract coordinates

            # only for vertical lines:
            angle = np.arctan2(abs(y1 - y2), abs(x1 - x2)) / np.pi * 180

            is_vert = angle > 80
            is_hor = angle < 10

            #[cell_n, cell_n  + 1]
            cell_n = x1 // int(cell_width)
            is_mispos_vert = min(abs(x1 - cell_n * cell_width), abs(x1 - (cell_n + 1) * cell_width)) > 0.2 * cell_width

            cell_n = y1 // int(cell_height)
            is_mispos_hor = min(abs(y1 - cell_n * cell_height), abs(y1 - (cell_n + 1) * cell_height)) > 0.2 * cell_width



            if (is_hor and not is_mispos_hor) or (is_vert and not is_mispos_vert):
                cv.line(mask, (x1, y1), (x2, y2), 255, 4)


    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, np.ones((3, 3)))

    return mask

root = "/home/user/Documents/data/sudoku-assistant/raw_data"
files = [
    "/home/user/Documents/data/sudoku-assistant/data/sudoku_1.jpeg",
    "/home/user/Documents/data/sudoku-assistant/data/sudoku_2.jpeg",
    "/home/user/Documents/data/sudoku-assistant/data/sudoku_3.jpeg"
]

# digits_dir = Path("/home/user/Documents/data/sudoku-assistant/digits")

# digits_dir.mkdir(parents=True, exist_ok=True)

for file in files:
    image = cv.imread(file)
    field_img = get_sudoku_field(image)
    #viz(field_img, file.name)

    mask = get_lines_mask(field_img)

    #viz(mask, 'mask')
    
    img_wo_grid = cv.bitwise_and(field_img, cv.bitwise_not(mask))

    img_wo_grid = cv.morphologyEx(img_wo_grid, cv.MORPH_CLOSE, kernel=np.ones((3, 3)))
    img_wo_grid = cv.erode(img_wo_grid, np.ones((3, 3)))
    #viz(img_wo_grid)
    viz(img_wo_grid)

    # height, width = img_wo_grid.shape

    # x_step = width / 9
    # y_step = height / 9

    # i = 0
    # for i_y, y in enumerate(np.arange(0, height, y_step)):
    #     for i_x, x in enumerate(np.arange(0, width, x_step)):

    #         m_y = 0.05 * y_step
    #         y1 = int(y + m_y)
    #         y2 = int(y + y_step - m_y)

    #         m_x = 0.05 * x_step
    #         x1 = int(x + m_x)
    #         x2 = int(x + x_step - m_x)

    #         roi = (img_wo_grid)[y1:y2, x1:x2]
    #         contours, _ = cv.findContours(roi, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    #         area_sorted = sorted(contours, key=cv.contourArea)

            
    #         if not len(area_sorted) or len(area_sorted) > 4 or cv.contourArea(area_sorted[-1]) < np.prod(roi.shape) * 0.05:
    #             print('Empty cells', (i_x, i_y))
    #         else:
    #             save = f"{file.stem}_d_{i_y}_{i_x}{file.suffix}"
    #             cv.imwrite(digits_dir / save, roi)


    